# Object Detection Demo: Car Detection

This is a sample reference implementation to showcase object detection (car in this case) with single-shot detection (SSD) and Async API.
Async API improves the overall frame-rate of the application by not waiting for the inference to complete but continuing to do things on the host while inference accelerator is busy. 
Specifically, this code demonstrates two parallel inference requests by processing the current frame while the next input frame is being captured. This essentially hides the latency of frame capture.

## Overview of how it works
At start-up the sample application reads the equivalent of command line arguments and loads a network and image from the video input to the Inference Engine (IE) plugin. 
A job is submitted to a hardware accelerator (Intel® Core CPU, Intel® HD Graphics GPU, Intel® Core CPU, Intel® Movidius™ Neural Compute Stick, Intel® Neural Compute Stick 2, Intel® HDDL-R, and Intel® HDDL-F.
After the inference is completed, the output videos are appropriately stored in the /results directory, which can then be viewed within the Jupyter Notebook instance.

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed on edge hardware (rather than on the development node hosting this Jupyter notebook)
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes
* Demonstrate the Async API in action


## Step 0: Set Up

### 0.1: Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *
from openvino.inference_engine import IEPlugin, IENetwork
import cv2
# For labeling the image
from out_process import placeBoxes

We will start with running inference on a single image to see how the Intel® Distribution of OpenVINO™ toolkit works, then we will run inference on a video stream.

We will go over detecting cars with OpenVINO in several steps:

1. Create an intermediate representation (IR) of the model using the Model Optimizer.
2. Create IEPlugin for the device.
3. Read the model's IR using IENetwork.
4. Load the IENetwork instance into the plugin.
5. Preprocess image and run inference on it.
6. Create a job file to target different hardware types.
7. Submit jobs to the queue.
8. View the results and hardware performance comparison.

## Create an Intermediate Representation of the Model

Model Optimizer creates the Intermediate Representation of the model which is the device-agnostic, generic optimization of the model. Caffe\*, TensorFlow\*, MXNet\*, ONNX\*, and Kaldi\* models are supported by Model Optimizer.

We will use the **MobileNet-SSD** model. Download the model, specifying the name and output directory.

In [ ]:
#Skip below command to download the models if you have problem to downlaod from network, local copy of models are under /raw_models
#!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o raw_models


Let's convert this model to the intermediate representation using Model Optimizer.

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP32 \
--output_dir models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

We will also need the FP16 IR version of the model for the calculations on Neural Compute Stick 2 (NCS2) and Visual Processing Unit (VPU); let's create it here.

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP16 \
--output_dir models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

## Create Plugin

Let's create a function to construct a plugin.

In [ ]:
def createPlugin(device, extension_list):
    # Plugin initialization for specified device. We will be targeting CPU initially.
    plugin = IEPlugin(device=device)

    # Loading additional extension libraries for the CPU
   # for extension in extension_list:
   #     plugin.add_cpu_extension('/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so')
    
    return plugin

## Read the IR of the Model

Let's import the optimized model into our neural network using **`IENetwork`**. 


In [ ]:
def createNetwork(model_xml, model_bin, plugin):
    # Importing network weights from IR models.
    net = IENetwork(model=model_xml, weights=model_bin)
    
    # Some layers in IR models may be unsupported by some plugins. 
    if "CPU" in plugin.device:
        supported_layers = plugin.get_supported_layers(net)
        not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
        if len(not_supported_layers) != 0:
            print("Following layers are not supported by the plugin for specified device {}:\n {}".
                      format(plugin.device, ', '.join(not_supported_layers)))
            print("Please try to specify cpu extensions library path in sample's command line parameters "
                  "using -l or --cpu_extension command line argument")
            return None
    return net

## Load the Network into the Plugin

Once we have the plugin and the network, we can load the network into the plugin.

In [ ]:
def loadNetwork(plugin, net):
    # Loading IR model to the plugin.
    exec_net = plugin.load(network=net, num_requests=2)
    
    # Getting the input and outputs of the network
    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    return exec_net,input_blob,out_blob

## Preprocess Image

We will need the function to load the image using OpenCV and change its shape to be compatible with our network.

In [ ]:
def preprocessImage(img_path, net, input_blob):
    # Reading the frame from a jpeg file
    frame = cv2.imread(img_path)
    
    # Reshaping data
    n, c, h, w = net.inputs[input_blob].shape
    in_frame = cv2.resize(frame, (w, h))
    in_frame = in_frame.transpose((2, 0, 1))  # Change data layout from HWC to CHW
    return in_frame.reshape((n, c, h, w)),frame

## Run Inference on a Single Frame

Now we are ready to run the inference workload using the plugin. We will be running in **async_mode** by using `start_async` method. With the async_mode, the inference is started in parallel on either a separate thread or device.
In other words, `start_async` is non-blocking and the main process is free to do any additional processing needed. 
In the next section, we will see an implementation of pipelining to mask the latency of loading and modifying images.

During asynchronous runs, the different images are tracked by an integer `request_id`. Because we only have one image to process, we will just use 0.

In [ ]:
# For labeling the image
from out_process import placeBoxes
import matplotlib.pyplot as plt

# Request id to keep track of
def runInference():
    plugin = createPlugin(device='CPU', extension_list=['libcpu_extension.so'])
    model_xml = "models/mobilenet-ssd/FP32/mobilenet-ssd.xml"
    model_bin = "models/mobilenet-ssd/FP32/mobilenet-ssd.bin"
    net = createNetwork(model_xml, model_bin, plugin)
    exec_net,input_blob,out_blob = loadNetwork(plugin, net)
    in_frame,original_frame = preprocessImage('cars_1900_first_frame.jpg', net, input_blob)
    
    my_request_id=0

    # Starting the inference in async mode, which starts the inference in parallel
    exec_net.start_async(request_id=my_request_id, inputs={input_blob: in_frame})
    # ... You can do additional processing or latency masking while we wait ...

    # Blocking wait for a particular request_id
    if exec_net.requests[my_request_id].wait(-1) == 0:
        # getting the result of the network
        res = exec_net.requests[my_request_id].outputs[out_blob]
        
        # Processing the output result and adding labels on the image. Implementation is not shown in the
        #  this notebook; you can find it in object_detection_demo_ssd_async.py
        prob_threshold = 0.5  # 50% confidence needed for "detection"
        initial_w = original_frame.shape[1]
        initial_h = original_frame.shape[0]
        frame = placeBoxes(res, None, prob_threshold, original_frame, initial_w, initial_h, False, my_request_id, 0)
        fig = plt.figure(dpi=300)
        ax = fig.add_subplot(111)
        ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), interpolation='none')
        plt.axis("off")
        plt.show()
    else:
        print("There was an error with the request")

runInference()

## Input Video

Run the following cell to create a symlink and view the input video.

In [ ]:
!ln -sf ./cars_1900.mp4 
videoHTML('Cars video', ['cars_1900.mp4'])

## How Video Inferencing Works

Video inferencing is similar to the image inferencing, however, there are few differences. Let's review them.

The following lines determine the source of the video. We will use a pre-recorded input video file in this example, we could also use a camera by setting the input argument to 'cam'.

```python
if args.input == 'cam':
        input_stream = 0
        out_file_name = 'cam'
    else:
        input_stream = args.input
```

We capture frames from the video sample using **OpenCV VideoCapture** API.

```python
cap = cv2.VideoCapture(input_stream)
```

Finally, we have a latency masking scheme, where we post-process a frames while other frames are being processed on the inference engine. User can control the number of inference requests running in parallel.

```python
current_inference = 0
required_inference_requests_were_executed = False
previous_inference = 1 - args.number_infer_requests
step = 0
steps_count = args.number_infer_requests - 1

while not required_inference_requests_were_executed or step < steps_count or cap.isOpened():
    # ... load the next frame from cap ...

    # start the next frame
    exec_net.start_async(request_id=current_inference, inputs={input_blob: in_frame})

    # see if the current frame is ready
    if previous_inference >= 0:
        status = infer_requests[previous_inference].wait()
    # ... post-processing current frame ...
    
    # manage request ids
    current_inference += 1
    if current_inference >= args.number_infer_requests:
        current_inference = 0
        required_inference_requests_were_executed = True

    previous_inference += 1
    if previous_inference >= args.number_infer_requests:
        previous_inference = 0

    step += 1
```

The Python code takes in command line arguments for video, model etc.

**Command line arguments options and how they are interpreted in the application source code**

```
SAMPLEPATH="/data/reference-sample-data"
python3 object_detection_demo_ssd_async.py -m ${SAMPLEPATH}/models/mobilenet-ssd/$3/mobilenet-ssd.xml \
                                           -i $INPUT_FILE \
                                           -o $RESULTS_PATH \
                                           -d $DEVICE \
                                           -nireq $NUM_INFER_REQS \
                                           -ce ${SAMPLEPATH}/extension/libcpu_extension.so
```

##### The description of the arguments used in the argument parser is the command line executable equivalent.
* -m location of the **mobilenet-ssd** pre-trained model which has been pre-processed using the **model optimizer**
   There is automated support built in this argument to support both FP32 and FP16 models targeting different hardware
   (**Note** we are using mobilenet-ssd in this example. However, OpenVINO's Inference Engine is compatible with other neural network architectures such as AlexNet*, GoogleNet*, SqueezeNet* etc.,)    

* -i location of the input video stream (video/cars_1900.mp4)
* -o location where the output file with inference needs to be stored. (results/core or results/xeon or results/gpu)
* -d Type of Hardware Acceleration (CPU or GPU or MYRIAD or HDDL or FPGA)
* -nireq Number of inference requests running in parallel
* -ce Absolute path to the shared library and is currently optimized for core/xeon (extension/libcpu_extension.so)


## Inference on a Video File

All the code up to this point has been run within the Jupyter Notebook instance running on a development node based on an Intel® Xeon® Scalable Processor, where the Notebook is allocated a single core. To run inference on the entire video, we need more compute power. We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

To pass the specific variables to the Python code, we will use following arguments:

* `-m`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the optimized **MobileNet-SSD** model's XML
* `-i`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the input video
* `-o`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output directory
* `-d`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;hardware device type (CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)
* `-l`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;path to the CPU extension library

The job file will be executed directly on the edge compute node.

In [ ]:

!python3 object_detection_demo_ssd_async.py  -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml \
                                            -i cars_1900.mp4 \
                                            -o results/CPU \
                                            -d CPU \
                                            -nireq 2 \
                                            -ce /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

!g++ -std=c++14 ROI_writer.cpp -o ROI_writer  -lopencv_core -lopencv_videoio -lopencv_videoio_ffmpeg -lopencv_imgproc -lopencv_highgui  -fopenmp -I/opt/intel/openvino/opencv/include/ -L/opt/intel/openvino/opencv/lib/
# Rendering the output video
#SKIPFRAME=1
#RESOLUTION=0.5
!./ROI_writer cars_1900.mp4 results/CPU 1 0.5

### Intel® Neural Compute Stick 2
The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
#Need to modify for Movidius VPU
#RUN inference on movidius VPU
#Progress indicators
!python3 object_detection_demo_ssd_async.py  -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml \
                                            -i cars_1900.mp4 \
                                            -o results/VPU \
                                            -d MYRIAD \
                                            -nireq 2 \
                                            -ce /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

!./ROI_writer cars_1900.mp4 results/VPU 1 0.5

***Wait!***

Please wait for the inference jobs and video rendering complete before proceeding to the next step.

## Step 3: View Results

Once the jobs are completed, the queue system outputs the `stdout` and `stderr` streams of each job into files with names
`obj_det_{type}.o{JobID}` and `obj_det_{type}.e{JobID}`. Here, obj_det_{type} corresponds to the `-N` option of qsub. For example, `core` for Core CPU target.

You can find the output video files inside the `results` directory. We wrote a short utility script that will display these videos within the notebook. See `demoutils.py` if you are interested in understanding further how the results are displayed in notebook. 

`obj_det_{type}.e{JobID}`

(here, obj_det_{type} corresponds to the `-N` option of qsub).

However, for this case, we may be more interested in the output video files. They are stored in mp4 format inside the `results/` directory.
We wrote a short utility script that will display these videos with in the notebook.
Run the cells below to display them.
See `demoutils.py` if you are interested in understanding further how the results are displayed in notebook.

In [ ]:
videoHTML('Intel Core CPU', 
          ['results/CPU/output_58593.c003.mp4'], 
          'results/CPU/stats_58593.c003.txt')

In [ ]:
#Need to modify for Movidius VPU

#videoHTML('IEI Tank + Intel CPU + Intel NCS2',
#          ['results/output_58593.c003.mp4',
#          'results/stats_58593.c003.txt')